## 데이터 준비하기
- prepare_custom_dataset
### directories and files for yours datsaets
/image_folder

├── 0_0_0000000

│   ├── 0_0.jpg

│   ├── 0_1.jpg

│   ├── 0_2.jpg

│   ├── 0_3.jpg

│   └── 0_4.jpg

├── 0_0_0000001

│   ├── 0_5.jpg

│   ├── 0_6.jpg

│   ├── 0_7.jpg

│   ├── 0_8.jpg

│   └── 0_9.jpg

In [ ]:
!pip install mxnet
!apt-get update
!apt-get install -y ffmpeg libsm6 libxext6

In [ ]:
!python -m mxnet.tools.im2rec --list --recursive train /content/drive/MyDrive/dataset
!python -m mxnet.tools.im2rec --num-thread 16 --quality 100 train /content/drive/MyDrive/dataset

In [ ]:
!pwd

In [ ]:
import shutil

source_path = '/content/train.idx'
destination_path = '/content/drive/MyDrive/딥러닝 (1)/team-project/arcface_torch/train_tmp/ms1m-retinaface-t1/train.idx'

shutil.move(source_path, destination_path)

In [ ]:
import shutil

source_path = '/content/train.lst'
destination_path = '/content/drive/MyDrive/딥러닝 (1)/team-project/arcface_torch/train_tmp/ms1m-retinaface-t1/train.lst'

shutil.move(source_path, destination_path)

In [ ]:
import shutil

source_path = '/content/train.rec'
destination_path = '/content/drive/MyDrive/딥러닝 (1)/team-project/arcface_torch/train_tmp/ms1m-retinaface-t1/train.rec'

shutil.move(source_path, destination_path)

# reference
## ArcFace
- https://github.com/deepinsight/insightface/tree/master/recognition/arcface_torch

In [ ]:
!pip install scipy==1.8.1
#설치하고 런타임 재시작해야됨

### import modules

In [3]:
!pip install mxnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
#arcface_torch 파일이 있는 경로
import sys
sys.path.append('/content/drive/MyDrive/딥러닝 (1)/team-project/arcface_torch')

In [5]:
import argparse
import logging
import os
from datetime import datetime

import numpy as np
import torch
from backbones import get_model
from dataset import get_dataloader
from losses import CombinedMarginLoss
from lr_scheduler import PolynomialLRWarmup
from partial_fc_v2 import PartialFC_V2
from torch import distributed
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from utils.utils_callbacks import CallBackLogging, CallBackVerification
from utils.utils_config import get_config
from utils.utils_distributed_sampler import setup_seed
from utils.utils_logging import AverageMeter, init_logging
from torch.distributed.algorithms.ddp_comm_hooks.default_hooks import fp16_compress_hook

assert torch.__version__ >= "1.12.0", "In order to enjoy the features of the new torch, \
we have upgraded the torch to 1.12.0. torch before than 1.12.0 may not work in the future."

try:
    rank = int(os.environ["RANK"])
    local_rank = int(os.environ["LOCAL_RANK"])
    world_size = int(os.environ["WORLD_SIZE"])
    distributed.init_process_group("nccl")
except KeyError:
    rank = 0
    local_rank = 0
    world_size = 1
    distributed.init_process_group(
        backend="nccl",
        init_method="tcp://127.0.0.1:12584",
        rank=rank,
        world_size=world_size,
    )

/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py:125: UserWarning: Decorating classes is deprecated and will be disabled in future versions. You should only decorate functions or methods. To preserve the current behavior of class decoration, you can directly decorate the `__init__` method and nothing else.
  warnings.warn("Decorating classes is deprecated and will be disabled in "


In [6]:
torch.backends.cudnn.benchmark = True

### config 파일 -> 'configs/ms1mv3_r50_onegpu'

- config.margin_list = (1.0, 0.5, 0.0)
- config.network = "r50" #backbone network
- config.resume = False
- config.output = None
- config.embedding_size = 512
- config.sample_rate = 1.0
- config.fp16 = True
- config.momentum = 0.9
- config.weight_decay = 5e-4
- config.batch_size = 128
- config.lr = 0.02
- config.verbose = 2000
- config.dali = False

- config.rec = "/train_tmp/ms1m-retinaface-t1" #dataset이 있는 경로
- config.num_classes = 93431 #수정 필요
- config.num_image = 5179510 # 수정 필요
- config.num_epoch = 20
- config.warmup_epoch = 0
- config.val_targets = ['lfw', 'cfp_fp', "agedb_30"]


In [7]:
cfg = get_config('configs/mydata_onegpu')

In [8]:
setup_seed(seed=cfg.seed, cuda_deterministic=False)

In [9]:
torch.cuda.set_device(local_rank)

os.makedirs(cfg.output, exist_ok=True)
init_logging(rank, cfg.output)

INFO:root:rank_id: 0


Training: 2023-06-17 05:57:26,611-rank_id: 0


In [10]:
summary_writer = (
        SummaryWriter(log_dir=os.path.join(cfg.output, "tensorboard"))
        if rank == 0
        else None
    )

### using_wandb

In [11]:
cfg.using_wandb

False

In [12]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
wandb_logger = None
if cfg.using_wandb:
    import wandb
    # Sign in to wandb
    try:
        wandb.login(key=cfg.wandb_key)
    except Exception as e:
        print("WandB Key must be provided in config file (base.py).")
        print(f"Config Error: {e}")
    # Initialize wandb
    run_name = datetime.now().strftime("%y%m%d_%H%M") + f"_GPU{rank}"
    run_name = run_name if cfg.suffix_run_name is None else run_name + f"_{cfg.suffix_run_name}"
    try:
        wandb_logger = wandb.init(
            entity = cfg.wandb_entity,
            project = cfg.wandb_project,
            sync_tensorboard = True,
            resume=cfg.wandb_resume,
            name = run_name,
            notes = cfg.notes) if rank == 0 or cfg.wandb_log_all else None
        if wandb_logger:
            wandb_logger.config.update(cfg)
    except Exception as e:
        print("WandB Data (Entity and Project name) must be provided in config file (base.py).")
        print(f"Config Error: {e}")

### trainloader(수정 필요)

In [14]:
#batch_size 크면 메모리 오류 뜸
cfg.batch_size

16

In [15]:
train_loader = get_dataloader(
    cfg.rec,
    local_rank,
    cfg.batch_size,
    cfg.dali,
    cfg.dali_aug,
    cfg.seed,
    cfg.num_workers
)

In [16]:
cfg.rec #dataset이 있는 경로

'/content/drive/MyDrive/딥러닝 (1)/team-project/arcface_torch/train_tmp/ms1m-retinaface-t1'

### backbone

In [17]:
backbone = get_model(
        cfg.network, dropout=0.0, fp16=cfg.fp16, num_features=cfg.embedding_size).cuda()

backbone = torch.nn.parallel.DistributedDataParallel(
    module=backbone, broadcast_buffers=False, device_ids=[local_rank], bucket_cap_mb=16,
    find_unused_parameters=True)
backbone.register_comm_hook(None, fp16_compress_hook)

backbone.train()
# FIXME using gradient checkpoint if there are some unused parameters will cause error
backbone._set_static_graph()

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/distributed.py:1915: UserWarning: You passed find_unused_parameters=true to DistributedDataParallel, `_set_static_graph` will detect unused parameters automatically, so you do not need to set find_unused_parameters=true, just be sure these unused parameters will not change during training loop while calling `_set_static_graph`.
  warnings.warn(


### margin loss

In [18]:
margin_loss = CombinedMarginLoss(
        64,
        cfg.margin_list[0],
        cfg.margin_list[1],
        cfg.margin_list[2],
        cfg.interclass_filtering_threshold
    )

### module_partial_fc

In [19]:
if cfg.optimizer == "sgd":
    module_partial_fc = PartialFC_V2(
        margin_loss, cfg.embedding_size, cfg.num_classes,
        cfg.sample_rate, False)
    module_partial_fc.train().cuda()
    # TODO the params of partial fc must be last in the params list
    opt = torch.optim.SGD(
        params=[{"params": backbone.parameters()}, {"params": module_partial_fc.parameters()}],
        lr=cfg.lr, momentum=0.9, weight_decay=cfg.weight_decay)

elif cfg.optimizer == "adamw":
    module_partial_fc = PartialFC_V2(
        margin_loss, cfg.embedding_size, cfg.num_classes,
        cfg.sample_rate, False)
    module_partial_fc.train().cuda()
    opt = torch.optim.AdamW(
        params=[{"params": backbone.parameters()}, {"params": module_partial_fc.parameters()}],
        lr=cfg.lr, weight_decay=cfg.weight_decay)
else:
    raise

### hyper parameters

In [20]:
cfg.total_batch_size = cfg.batch_size * world_size
cfg.warmup_step = cfg.num_image // cfg.total_batch_size * cfg.warmup_epoch
cfg.total_step = cfg.num_image // cfg.total_batch_size * cfg.num_epoch

lr_scheduler = PolynomialLRWarmup(
    optimizer=opt,
    warmup_iters=cfg.warmup_step,
    total_iters=cfg.total_step)

start_epoch = 0
global_step = 0

In [21]:
cfg.total_batch_size

16

In [22]:
cfg.warmup_step

0

### checkpoint

#### 학습 다시 재개하고 싶을 때

In [23]:
cfg.resume

False

In [24]:
if cfg.resume:
    dict_checkpoint = torch.load(os.path.join(cfg.output, f"checkpoint_gpu_{rank}.pt"))
    start_epoch = dict_checkpoint["epoch"]
    global_step = dict_checkpoint["global_step"]
    backbone.module.load_state_dict(dict_checkpoint["state_dict_backbone"])
    module_partial_fc.load_state_dict(dict_checkpoint["state_dict_softmax_fc"])
    opt.load_state_dict(dict_checkpoint["state_optimizer"])
    lr_scheduler.load_state_dict(dict_checkpoint["state_lr_scheduler"])
    del dict_checkpoint


In [25]:
for key, value in cfg.items():
    num_space = 25 - len(key)
    logging.info(": " + key + " " * num_space + str(value))

INFO:root:: margin_list              [1.0, 0.5, 0.0]


Training: 2023-06-17 05:58:25,726-: margin_list              [1.0, 0.5, 0.0]


INFO:root:: network                  r50


Training: 2023-06-17 05:58:25,730-: network                  r50


INFO:root:: resume                   False


...

Training: 2023-06-17 05:58:25,810-: total_batch_size         16


INFO:root:: warmup_step              0


Training: 2023-06-17 05:58:25,812-: warmup_step              0


INFO:root:: total_step               47420


Training: 2023-06-17 05:58:25,814-: total_step               47420


In [26]:
callback_verification = CallBackVerification(
    val_targets=cfg.val_targets, rec_prefix=cfg.rec,
    summary_writer=summary_writer, wandb_logger = wandb_logger
)
callback_logging = CallBackLogging(
    frequent=cfg.frequent,
    total_step=cfg.total_step,
    batch_size=cfg.batch_size,
    start_step = global_step,
    writer=summary_writer
)

loss_am = AverageMeter()
amp = torch.cuda.amp.grad_scaler.GradScaler(growth_interval=100)

loading bin 0
loading bin 1000
loading bin 2000
loading bin 3000
loading bin 4000
loading bin 5000
loading bin 6000
loading bin 7000
loading bin 8000
loading bin 9000
loading bin 10000
loading bin 11000
torch.Size([12000, 3, 112, 112])
loading bin 0
loading bin 1000
loading bin 2000
loading bin 3000
loading bin 4000
loading bin 5000
loading bin 6000
loading bin 7000
loading bin 8000
loading bin 9000
loading bin 10000
loading bin 11000
loading bin 12000
loading bin 13000
torch.Size([14000, 3, 112, 112])
loading bin 0
loading bin 1000
loading bin 2000
loading bin 3000
loading bin 4000
loading bin 5000
loading bin 6000
loading bin 7000
loading bin 8000
loading bin 9000
loading bin 10000
loading bin 11000
torch.Size([12000, 3, 112, 112])


In [27]:
callback_verification.ver_name_list

['lfw', 'cfp_fp', 'agedb_30']

### train

In [28]:
for _, (img, local_labels) in enumerate(train_loader):
  break
img.size()

torch.Size([16, 3, 112, 112])

In [29]:
cfg.output

'work_dirs/mydata_onegpu'

In [30]:
cfg.num_epoch

20

In [39]:
import scipy
print(scipy.__version__)

1.10.1


In [40]:
#degrade scipy (callback_verification(global_step, backbone) 여기서 ValueError: Expect x to not have duplicates 발생)
!pip install scipy==1.8.1
#설치하고 런타임 재시작해야됨

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
#degrade 됐는지 확인
import scipy
print(scipy.__version__)

1.8.1


In [32]:
for epoch in range(start_epoch, cfg.num_epoch):

    if isinstance(train_loader, DataLoader):
        train_loader.sampler.set_epoch(epoch)
    for _, (img, local_labels) in enumerate(train_loader):
        global_step += 1
        local_embeddings = backbone(img)
        loss: torch.Tensor = module_partial_fc(local_embeddings, local_labels)

        if cfg.fp16:
            amp.scale(loss).backward()
            if global_step % cfg.gradient_acc == 0:
                amp.unscale_(opt)
                torch.nn.utils.clip_grad_norm_(backbone.parameters(), 5)
                amp.step(opt)
                amp.update()
                opt.zero_grad()
        else:
            loss.backward()
            if global_step % cfg.gradient_acc == 0:
                torch.nn.utils.clip_grad_norm_(backbone.parameters(), 5)
                opt.step()
                opt.zero_grad()
        lr_scheduler.step()

        with torch.no_grad():
            if wandb_logger:
                wandb_logger.log({
                    'Loss/Step Loss': loss.item(),
                    'Loss/Train Loss': loss_am.avg,
                    'Process/Step': global_step,
                    'Process/Epoch': epoch
                })

            loss_am.update(loss.item(), 1)
            callback_logging(global_step, loss_am, epoch, cfg.fp16, lr_scheduler.get_last_lr()[0], amp)

            if global_step % cfg.verbose == 0 and global_step > 0:
                callback_verification(global_step, backbone)

    if cfg.save_all_states:
        checkpoint = {
            "epoch": epoch + 1,
            "global_step": global_step,
            "state_dict_backbone": backbone.module.state_dict(),
            "state_dict_softmax_fc": module_partial_fc.state_dict(),
            "state_optimizer": opt.state_dict(),
            "state_lr_scheduler": lr_scheduler.state_dict()
        }
        torch.save(checkpoint, os.path.join(cfg.output, f"checkpoint_gpu_{rank}.pt"))

    if rank == 0:
        path_module = os.path.join(cfg.output, "model.pt")
        torch.save(backbone.module.state_dict(), path_module)

        if wandb_logger and cfg.save_artifacts:
            artifact_name = f"{run_name}_E{epoch}"
            model = wandb.Artifact(artifact_name, type='model')
            model.add_file(path_module)
            wandb_logger.log_artifact(model)

    if cfg.dali:
        train_loader.reset()

if rank == 0:
    path_module = os.path.join(cfg.output, "model.pt")
    torch.save(backbone.module.state_dict(), path_module)

    if wandb_logger and cfg.save_artifacts:
        artifact_name = f"{run_name}_Final"
        model = wandb.Artifact(artifact_name, type='model')
        model.add_file(path_module)
        wandb_logger.log_artifact(model)

Output hidden; open in https://colab.research.google.com to view.

In [33]:
import shutil

source_path = '/content/work_dirs'
destination_path = '/content/drive/MyDrive/딥러닝 (1)/team-project/arcface_torch/wrok_dirs'

shutil.move(source_path, destination_path)

'/content/drive/MyDrive/딥러닝 (1)/team-project/arcface_torch/wrok_dirs'